In [1]:
import sys
sys.path.append('/Users/apple/Desktop/AiCore Bootcamp/multinational-retail-data-centralisation939')

In [2]:
from src.database_utils import DatabaseConnector
from src.data_cleaning import DataCleaning
from src.data_extraction import DataExtractor
from src.data_extraction import header
import numpy as np
import pandas as pd

In [3]:
# Read the credentials yaml file and return a dictionary of the credentials.
instance = DatabaseConnector('../../db_creds.yaml')
db_creds = instance.read_db_creds()

In [4]:
# Read the credentials from the return of read_db_creds and initialise and return an sqlalchemy database engine.
db_engine = instance.init_db_engine()

In [5]:
# Create a method list_db_tables to list all the tables in the database so you know which tables you can extract data from.
table_names = instance.list_db_tables()
print(table_names)

['legacy_store_details', 'legacy_users', 'orders_table']


In [6]:
# Develop a method called read_rds_table in your DataExtractor class which will extract the database table to a pandas DataFrame.
extractor = DataExtractor()
user_data = extractor.read_rds_table('legacy_users')
user_data.head()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
index,,,,,,,,,,,
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [7]:
# Missing values before cleaning
missing_values_per_column = user_data.isnull().sum()
print(missing_values_per_column)

first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64


In [8]:
user_data.shape[0]

15320

In [9]:
# Clean data
cleaned_user_data = DataCleaning().clean_user_data(user_data)

In [11]:
cleaned_user_data.head()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
index,,,,,,,,,,,
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,49047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,01614960674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,441214960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,03069990871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,01214960225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [12]:
# Missing values after cleaning
missing_values_per_column = cleaned_user_data.isnull().sum()
print(missing_values_per_column)

first_name       16
last_name        15
date_of_birth    16
company          16
email_address    59
address          16
country          16
country_code     16
phone_number      0
join_date        16
user_uuid        16
dtype: int64


In [13]:
# Upload to SQL in dim_users table
instance = DatabaseConnector('../../sales_data_creds.yaml')
instance.upload_to_db(cleaned_user_data, 'dim_users')

300